<a href="https://colab.research.google.com/github/AlexeyTri/PROJECT_SCOOL/blob/master/ActualGraphicBASE_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Условия:**
1. наличие AUTODESK DESKTOP
2. доступ ко всем таблицам в режиме РЕДАКТОР
3. все тактические графики должны быть приведены в гугл форму

# Библиотеки

In [47]:
!pip install pygsheets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
import pygsheets

In [49]:
import openpyxl
from openpyxl.cell.cell import WriteOnlyCell
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
import pandas as pd
import gspread_dataframe as gd

In [90]:
idx = pd.IndexSlice

In [51]:
from google.colab import auth
auth.authenticate_user()
import re
import gspread
from google.auth import default
creds, _ = default()
import numpy as np
gc = gspread.authorize(creds)
from datetime import datetime

# отчетные даты

In [139]:
# старт-финиш проекта
# start_project = '01.02.2022'
start_project = '2022-02-01'
# start_project = pd.to_datetime(start_project)
# finish_project = '02.09.2024'
finish_project = '2024-09-02'
# finish_project = pd.to_datetime(finish_project)

In [140]:
# страт-финиш отчетной 
# start_week = '04.03.2023'
start_week = '2023-03-04'
# start_week = pd.to_datetime(start_week)
# finish_week = '20.03.2023'
finish_week = '2023-03-20'
# finish_week = pd.to_datetime(finish_week)

In [141]:
# старт-финиш next недели, за отчетной
# start_next_week = '21.03.2023'
start_next_week = '2023-03-21'
# start_next_week = pd.to_datetime(start_next_week)
# finish_next_week = '31.03.2023'
finish_next_week = '2023-03-31'
# finish__next_week = pd.to_datetime(finish_next_week)

# Предобработка GS: график проектирования, график СМР, график заказчика

## Реестр выдачи РД

https://docs.google.com/spreadsheets/d/1ZuZv1Rx-mp3AS5LFPjbEa-P92Xux_o_rvaOQb6NWXkE/edit#gid=479522789

In [145]:
# форма для обновления графика заказчика
worksheet_PD = gc.open('Реестр выдачи РД')
graph_PD = worksheet_PD.worksheet('График ПИР ')
graph_PD = graph_PD.get_all_values()
graph_PD = np.array(graph_PD)
graph_PD = pd.DataFrame(graph_PD[7:])
graph_PD

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,,,,Корректировка Проектной документации,,,09.12.2022,,,,,,,,,,,
1,1,,,Общая пояснительная записка,"ООО ""ИБС""",24.06.2022,08.12.2022,STATUS B,FALSE,FALSE,TRUE,1,1,0,0,0,,
2,2,,,Схема планировочной организации земельного уча...,"ООО ""ИБС""",24.06.2022,21.11.2022,STATUS B,FALSE,FALSE,TRUE,1,1,0,0,0,,
3,3,,,Архитектурные решения,"ООО ""ИБС""",24.06.2022,09.12.2022,STATUS B,FALSE,FALSE,TRUE,1,1,0,0,0,,
4,4,,,Конструктивные решения ГЧ,"ООО ""ИБС""",24.06.2022,25.11.2022,STATUS B,FALSE,FALSE,TRUE,1,1,0,0,0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,227,,,Первчиное консультирование по СМР проведение т...,"ООО ""Экостандарт""",06.10.2022,20.11.2022,STATUS A,FALSE,TRUE,FALSE,1,1,0,0,0,,
244,228,,,Сопровождение строительства,"ООО ""Экостандарт""",06.10.2022,06.05.2024,HOLD,TRUE,FALSE,FALSE,0,1,0,406,399,,
245,229,,,Отчет по вводу объекта в эксплуатацию,"ООО ""Экостандарт""",06.05.2024,03.06.2024,HOLD,TRUE,FALSE,FALSE,0,1,0,434,427,,
246,230,,,Энергетическое моделирование,"ООО ""Экостандарт""",20.06.2023,31.10.2023,HOLD,TRUE,FALSE,FALSE,0,1,0,218,211,,


In [146]:
# форма для обновления графика подрядчика
graph_PD_RD = pd.DataFrame.from_records(worksheet_PD.worksheet('График ПИР ').get_all_values())
graph_PD_RD

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,,,78,Просрочено,,START,FINISH,,,,,,,,,,,
1,,,5,До конца срока менее 1 недели,,,,,,,,,,,,,,
2,,,18,До конца срока менее 2-х недель,,,,,,,,,,,,,,
3,,,102,Выполнено,,,,,,,,,,,,,,
4,,20.03.2023,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,227,,,Первчиное консультирование по СМР проведение т...,"ООО ""Экостандарт""",06.10.2022,20.11.2022,STATUS A,FALSE,TRUE,FALSE,1,1,0,0,0,,
251,228,,,Сопровождение строительства,"ООО ""Экостандарт""",06.10.2022,06.05.2024,HOLD,TRUE,FALSE,FALSE,0,1,0,406,399,,
252,229,,,Отчет по вводу объекта в эксплуатацию,"ООО ""Экостандарт""",06.05.2024,03.06.2024,HOLD,TRUE,FALSE,FALSE,0,1,0,434,427,,
253,230,,,Энергетическое моделирование,"ООО ""Экостандарт""",20.06.2023,31.10.2023,HOLD,TRUE,FALSE,FALSE,0,1,0,218,211,,


## План работ Договор Адамант

https://docs.google.com/spreadsheets/d/1ngWR2k38yG5qDWfwA9LKi4-R9q9PrIHludYb5vSh4Gc/edit#gid=1411311560

In [147]:
# форма для обновления графика заказчика
worksheet_SMR = gc.open('План работ Договор Адамант')
graf = worksheet_SMR.worksheet("ГРАФИК РАБОТ")
graf = graf.get_all_values()
graf = np.array(graf)
graf =  pd.DataFrame(graf[1:, :])
graf

,0,1,2,3,4,5,6,7,8,9,...,420,421,422,423,424,425,426,427,428,429
0,PLAN,ГРАФИК РАБОТ,Начало работ по ГРП,Завершение работ по ГРП,Длительность,Ед. изм.,Объем по контракту,План на 20.03,Факт на 20.03,% выполнения плана,...,03.05.2023,03.05.2023,04.05.2023,04.05.2023,05.05.2023,05.05.2023,06.05.2023,06.05.2023,07.05.2023,07.05.2023
1,***,,,,,,,,,,...,план,факт,план,факт,план,факт,план,факт,план,факт
2,VZO_POS1_POD_MOB_.,Мобилизация,10.10.2022,10.05.2024,579,--,,,,"82,61%",...,,,,,,,,,,
3,VZO_POS1_POD_VRD_DP17.01,Временные дороги из дорожных плит (3х1.75),26.10.2022,18.02.2023,116,м2,300,"300,0","300,0","100,00%",...,,,,,,,,,,
4,VZO_POS1_POD_VRS_ES0.01,Временные электросети - установленные на начал...,23.11.2022,12.12.2022,20,комплект,2,"2,0","1,0","50,00%",...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,BOT_GP___.,Работы по благоустройству,18.11.2022,05.04.2024,505,,,"0,0","0,0",,...,,,,,,,,,,
382,BOT_GP,Выдача рабочего проекта по разделу,18.11.2022,18.11.2022,1,РД,,,,,...,,,,,,,,,,
383,_EXP___.,Ввод объекта в эксплуатацию,14.03.2024,06.06.2024,85,,,"0,0","0,0",,...,,,,,,,,,,
384,_ZOS___.,ЗОС,14.03.2024,24.04.2024,42,,,"0,0","0,0",,...,,,,,,,,,,


In [148]:
# форма для заполнения факта
graf_black = graf.copy()

In [149]:
# форма для обновления графика недели
week_graf = graf.copy()
week_graf = week_graf.loc[idx[2:],idx[:6]]
week_graf.drop([4,5], axis=1, inplace=True)
week_graf.rename(columns = {0:'passwordCOST', 
                       1:'NAME',2:'START', 3:'FINISH', 6:'valueModel'}, 
            inplace = True)
week_graf = week_graf[week_graf['passwordCOST'] > '']
week_graf = week_graf[week_graf['START'] > '']
week_graf = week_graf[week_graf['FINISH'] > '']
week_graf = week_graf.set_index('passwordCOST')
week_graf['valueModel'] = week_graf['valueModel'].map(lambda x: pd.to_numeric(re.sub("[^0-9+,]", "", x).replace(',','.')))
week_graf = week_graf.fillna(0)
week_graf

,NAME,START,FINISH,valueModel
passwordCOST,,,,
VZO_POS1_POD_MOB_.,Мобилизация,10.10.2022,10.05.2024,0.0
VZO_POS1_POD_VRD_DP17.01,Временные дороги из дорожных плит (3х1.75),26.10.2022,18.02.2023,300.0
VZO_POS1_POD_VRS_ES0.01,Временные электросети - установленные на начал...,23.11.2022,12.12.2022,2.0
VZO_POS1_POD_VRS_VK0.01,Временный водопровод и канализация (с подогрев...,19.10.2022,14.11.2022,1.0
VZO_POS1_POD_VRD_DP17.02,Устройство основания под бытовые помещения (по...,16.10.2022,09.11.2022,200.0
...,...,...,...,...
BOT_GP___.,Работы по благоустройству,18.11.2022,05.04.2024,0.0
BOT_GP,Выдача рабочего проекта по разделу,18.11.2022,18.11.2022,0.0
_EXP___.,Ввод объекта в эксплуатацию,14.03.2024,06.06.2024,0.0


In [150]:
# общий график работ
graf_name = graf[[0,1]][2:]
graf_name.rename(columns={0: 'passwordCOST', 1: 'NAME'}, inplace=True) 
graf_name = graf_name[graf_name['passwordCOST'] != 0]
graf_name = pd.DataFrame(graf_name['NAME'].values, index=graf_name['passwordCOST'], columns=['NAME'])
graf_total = graf.drop(graf.index[1:10], axis=1)
newColumnsIndex = [pd.to_datetime(pd.Series(graf_total.iloc[0].values[1:]).apply(lambda x: str(x)), format='%d.%m.%Y'), graf_total.iloc[1][1:].values]
graf_total.rename(columns={0: 'passwordCOST'}, inplace=True)
graf_total = graf_total.set_index("passwordCOST")
graf_total = pd.DataFrame(graf_total.values, index=graf_total.index, columns=newColumnsIndex)
graf_total.drop(graf_total.index[:2], axis=0, inplace=True)
for i in graf_total.columns:
    graf_total[i] = graf_total[i].map(lambda x: pd.to_numeric(re.sub("[^0-9 +,]", "", x).replace(',','.')))
graf_total.fillna(0, inplace=True)
graf_total.sort_index(axis=0, level=1, ascending=True)
graf_total = graf_total[graf_total.index > '']
graf_total

2022-10-10      2022-10-11      2022-10-12       \
                               план факт       план факт       план факт   
passwordCOST                                                               
VZO_POS1_POD_MOB_.              0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRD_DP17.01        0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRS_ES0.01         0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRS_VK0.01         0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRD_DP17.02        0.0  0.0        0.0  0.0        0.0  0.0   
...                             ...  ...        ...  ...        ...  ...   
BOT_GP___.                      0.0  0.0        0.0  0.0        0.0  0.0   
BOT_GP                          0.0  0.0        0.0  0.0        0.0  0.0   
_EXP___.                        0.0  0.0        0.0  0.0        0.0  0.0   
_ZOS___.                        0.0  0.0        0.0  0.0        0.0  0.0   
_RNV___.                        0.0  0.0        0.0  0.0        0.0  0.0   

                         2022-10-13      2022-10-14       ... 2023-05-03       \
                               план факт       план факт  ...       план факт   
passwordCOST                                              ...                   
VZO_POS1_POD_MOB_.              0.0  0.0        0.0  0.0  ...        0.0  0.0   
VZO_POS1_POD_VRD_DP17.01        0.0  0.0        0.0  0.0  ...        0.0  0.0   
VZO_POS1_POD_VRS_ES0.01         0.0  0.0        0.0  0.0  ...        0.0  0.0   
VZO_POS1_POD_VRS_VK0.01         0.0  0.0        0.0  0.0  ...        0.0  0.0   
VZO_POS1_POD_VRD_DP17.02        0.0  0.0        0.0  0.0  ...        0.0  0.0   
...                             ...  ...        ...  ...  ...        ...  ...   
BOT_GP___.                      0.0  0.0        0.0  0.0  ...        0.0  0.0   
BOT_GP                          0.0  0.0        0.0  0.0  ...        0.0  0.0   
_EXP___.                        0.0  0.0        0.0  0.0  ...        0.0  0.0   
_ZOS___.                        0.0  0.0        0.0  0.0  ...        0.0  0.0   
_RNV___.                        0.0  0.0        0.0  0.0  ...        0.0  0.0   

                         2023-05-04      2023-05-05      2023-05-06       \
                               план факт       план факт       план факт   
passwordCOST                                                               
VZO_POS1_POD_MOB_.              0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRD_DP17.01        0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRS_ES0.01         0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRS_VK0.01         0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRD_DP17.02        0.0  0.0        0.0  0.0        0.0  0.0   
...                             ...  ...        ...  ...        ...  ...   
BOT_GP___.                      0.0  0.0        0.0  0.0        0.0  0.0   
BOT_GP                          0.0  0.0        0.0  0.0        0.0  0.0   
_EXP___.                        0.0  0.0        0.0  0.0        0.0  0.0   
_ZOS___.                        0.0  0.0        0.0  0.0        0.0  0.0   
_RNV___.                        0.0  0.0        0.0  0.0        0.0  0.0   

                         2023-05-07       
                               план факт  
passwordCOST                              
VZO_POS1_POD_MOB_.              0.0  0.0  
VZO_POS1_POD_VRD_DP17.01        0.0  0.0  
VZO_POS1_POD_VRS_ES0.01         0.0  0.0  
VZO_POS1_POD_VRS_VK0.01         0.0  0.0  
VZO_POS1_POD_VRD_DP17.02        0.0  0.0  
...                             ...  ...  
BOT_GP___.                      0.0  0.0  
BOT_GP                          0.0  0.0  
_EXP___.                        0.0  0.0  
_ZOS___.                        0.0  0.0  
_RNV___.                        0.0  0.0  

[366 rows x 420 columns]

In [151]:
# перечень работ -> вгрузка из Plan Radar
plan_PR = worksheet_SMR.worksheet('PR')
plan_PR = plan_PR.get_all_values()
plan_PR = np.array(plan_PR)
plan_PR = pd.DataFrame(plan_PR)
plan_PR = plan_PR.iloc[1:, [0,7,9]]
plan_PR = pd.DataFrame(plan_PR)
plan_PR

,0,7,9
1,VZO_POS1_POD_INF_STN.01,1.0,2022-10-27
2,OOS_KR1_ZEM_VP_00.01,1.0,2022-11-02
3,VZO_POS1_POD_VRD_DP17.01,140.0,2022-11-02
4,VZO_POS1_POD_VRD_DP17.02,80.0,2022-11-02
5,VZO_POS1_POD_VRS_VK0.01,1.0,2022-11-02
...,...,...,...
156,OOS_KZ01_ZEM_OZ_PAZ.01,596.77,2023-03-07
157,OOS_KR1_IZO_GWL_PGE.01,130.0,2023-03-07
158,OOS_KR3_IZO_GWL_MEM.01,800.0,2023-03-08
159,OOS_KR3_IZO_GWL_STU.01,28.0,2023-03-09


## График по форме заказчика

In [ ]:
# файл загружается на ГУГЛ диск
from google.colab import files
upload = files.upload()

Saving Сводный план-график 2023 03 03 (1).xlsx to Сводный план-график 2023 03 03 (1).xlsx


In [152]:
df_graph = pd.read_excel(open('Сводный план-график 2023 03 03 (1).xlsx', 'rb'))
df_graph

/usr/local/lib/python3.9/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Рекомендации по фильтру:,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,Статус задач на 20.02.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1. В фильтре по столбцу ""Названия ресурсов"" на...",NaN,NaN,NaN,1. Уровень. Группа
1,NaN,NaN,КТ1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Уровень. Суммарная задача
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Внесено изменение,СДР,Название задачи,Нарушен срок,%,План %,Откл. План%,Столбец2,Столбец1,Начало,Окончание,Отв. Исполнитель,Названия ресурсов,Столбец3,Столбец4
4,NaN,0,Проект Школа Новый Вгляд 2022 12 27,0,0.21,24,3,Вт 18.01.22,Ср 10.12.25,2022-01-18 00:00:00,2025-12-10 00:00:00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,NaN,16.3.10,ПО бухгалтерского и финансово-экономи...,0,0,0,0,Пн 30.01.23,Ср 01.03.23,2023-01-30 00:00:00,2023-03-01 00:00:00,Операционный директор,Операционный директор,NaN,NaN
1251,NaN,16.3.11,ПО зарплаты и управления персоналом,0,0,0,0,Пн 30.01.23,Ср 01.03.23,2023-01-30 00:00:00,2023-03-01 00:00:00,Операционный директор,Операционный директор,NaN,NaN
1252,NaN,16.3.12,ПО автоматизации медицинского пункта,0,0,0,0,Пн 30.01.23,Ср 01.03.23,2023-01-30 00:00:00,2023-03-01 00:00:00,Операционный директор,Операционный директор,NaN,NaN
1253,NaN,16.3.13,ПО системы заказа пропусков,0,0,0,0,Пн 30.01.23,Ср 01.03.23,2023-01-30 00:00:00,2023-03-01 00:00:00,Операционный директор,Операционный директор,NaN,NaN


## Реестр замечаний из ACC

In [ ]:
# файл с реестром замечаний на google drive
from google.colab import files
upload = files.upload()

Saving Сводка по проблемам EXCEL-202303200933.xlsx to Сводка по проблемам EXCEL-202303200933.xlsx


In [153]:
value_essuse = pd.read_excel(open('Сводка по проблемам EXCEL-202303200933.xlsx', 'rb'), sheet_name=1)
value_essuse = value_essuse[(value_essuse['Статус'] == 'Открыто') & (value_essuse['Тип'].isin(['Замечание к РД','Замечания к ПД']))]
value_essuse.loc[:, 'Размещение'] =  value_essuse['Размещение'].map(lambda x: str(x).split('.pdf')[0].split('_')[0])
value_essuse = value_essuse.loc[:, idx['Размещение','Вид замечания']]
agg_func_count = {'Вид замечания': 'count'}
value_essuse = value_essuse.groupby(['Размещение','Вид замечания']).agg(agg_func_count)
value_essuse

Вид замечания
Размещение       Вид замечания                       
608AR-20-Р-АСКУЭ Критическое замечание              4
                 Недостаточно данных                4
                 Оформление                         1
608AR-20-Р-ДВТ   Критическое замечание              8
                 Оформление                         1
608AR-20-Р-ИБ    Оформление                         1
608AR-20-Р-ИТ    Оформление                         1
608AR-20-Р-КЖ-14 Техническая ошибка                 1
608AR-20-Р-СВДС  Критическое замечание              6
                 Недостаточно данных                1

# **Обновление графика проектирования подрядчика: 'График ПИР '**

* выгрузка реестра замечаний 
* агрегирование данных по разделам
* заполнение графика проектирования

In [154]:
name_RD_issues = pd.Series(value_essuse.index.get_level_values(0)).unique()

In [155]:
graph_PD_RD = pd.DataFrame.from_records(worksheet_PD.worksheet('График ПИР ').get_all_values())

In [156]:
list_PD_RD = graph_PD_RD.iloc[:,2].to_list()

In [157]:
for i in name_RD_issues:
    try:
        number_row = list_PD_RD.index(i) + 1
        try:
            value_essuse.loc[i, 'Критическое замечание']
            worksheet_PD.worksheet('График ПИР ').update_cell(number_row, 8, 'STATUS C')
        except: 
            try:
                value_essuse.loc[i, 'Техническая ошибка'].values[0] != 0  
                value_essuse.loc[i, 'Недостаточно данных'].values[0] != 0
                worksheet_PD.worksheet('График ПИР ').update_cell(number_row, 8, 'STATUS B')
            except:
                try:
                    worksheet_PD.worksheet('График ПИР ').update_cell(number_row, 8, 'STATUS A') 
                except:
                    pass
    except:
        pass 

# **Обновление графика СМР подрядчика: 'PLAN_WEEK', 'START', 'ГРАФИК РАБОТ'**

* в модели отмечается выполненный элемент
* данные попадают в GS + обновляется формы PLAN RADAR

## срез на отчетную неделю, план на следующую неделю за отчетной, кумулятивная сумма

In [158]:
graf_week_plan = graf_total.loc[:, idx[start_week:finish_week,'план']].sum(axis=1).values
graf_week_plan = pd.DataFrame(graf_week_plan,index=graf_total.index, columns=['PLAN_WEEK'])
graf_week_fakt = graf_total.loc[:, idx[start_week:finish_week,'факт']].sum(axis=1).values
graf_week_fakt = pd.DataFrame(graf_week_fakt,index=graf_total.index, columns=['FAKT_WEEK'])

In [161]:
 graf_total[graf_total.index == 'OOS_KZ03_BK1_BWL_OAB30.01'].loc[:, idx[start_week:finish_week,'факт']]

,2023-03-04,2023-03-05,2023-03-06,2023-03-07,2023-03-08,2023-03-09,2023-03-10,2023-03-11,2023-03-12,2023-03-13,2023-03-14,2023-03-15,2023-03-16,2023-03-17,2023-03-18,2023-03-19,2023-03-20
,факт,факт,факт,факт,факт,факт,факт,факт,факт,факт,факт,факт,факт,факт,факт,факт,факт
passwordCOST,,,,,,,,,,,,,,,,,
OOS_KZ03_BK1_BWL_OAB30.01,5.4,12.9,28.7,18.7,16.0,4.7,0.0,31.2,12.0,0.0,0.0,33.0,9.0,51.0,29.4,0.0,0.0
OOS_KZ03_BK1_BWL_OAB30.01,26.0,14.5,31.0,19.3,20.0,12.5,0.0,33.5,8.0,3.0,0.0,29.6,12.0,34.0,20.0,2.3,0.0


In [138]:
graf_total[graf_total.index == 'OOS_KZ03_BK1_BWL_OAB30.01'].loc[:, idx['2023-03-03':'2023-03-20','план']]

,2023-03-03,2023-03-04,2023-03-05,2023-03-06,2023-03-07,2023-03-08,2023-03-09,2023-03-10,2023-03-11,2023-03-12,2023-03-13,2023-03-14,2023-03-15,2023-03-16,2023-03-17,2023-03-18,2023-03-19,2023-03-20
,план,план,план,план,план,план,план,план,план,план,план,план,план,план,план,план,план,план
passwordCOST,,,,,,,,,,,,,,,,,,
OOS_KZ03_BK1_BWL_OAB30.01,12.3,12.3,12.3,12.3,12.3,12.3,12.3,12.3,12.3,12.3,12.3,12.3,12.3,12.3,12.3,0.0,0.0,0.0
OOS_KZ03_BK1_BWL_OAB30.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [164]:
graf_next_week_plan = graf_total.loc[:, idx[start_next_week:finish_next_week,'план']].sum(axis=1).values
graf_next_week_plan = pd.DataFrame(graf_next_week_plan,index=graf_total.index, columns=['PLAN_NEXT_WEEK'])
graf_next_week_fakt = graf_total.loc[:, idx[start_next_week:finish_next_week,'факт']].sum(axis=1).values
graf_next_week_fakt = pd.DataFrame(graf_next_week_fakt,index=graf_total.index, columns=['FAKT_NEXT_WEEK'])

In [166]:
total_plan = graf_total.loc[:, idx[start_project:finish_week,'план']].sum(axis=1).values
total_plan = pd.DataFrame(total_plan,index=graf_total.index, columns=['TOTAL_PLAN'])
total_fakt = graf_total.loc[:, idx[start_project:finish_week,'факт']].sum(axis=1).values
total_fakt = pd.DataFrame(total_fakt,index=graf_total.index, columns=['TOTAL_FAKT'])

In [167]:
new_graf_week = week_graf.join([total_plan,total_fakt,graf_week_plan,graf_week_fakt,graf_next_week_plan,graf_next_week_fakt])

In [168]:
new_graf_week = new_graf_week[['NAME', 'TOTAL_PLAN','TOTAL_FAKT','PLAN_WEEK','FAKT_WEEK','PLAN_NEXT_WEEK','FAKT_NEXT_WEEK']]

In [169]:
new_graf_week = new_graf_week.loc[(new_graf_week['PLAN_WEEK']>0) | (new_graf_week['PLAN_NEXT_WEEK']>0)]

In [170]:
new_graf_week['%TOTAl_WEEK'] =( new_graf_week['FAKT_WEEK'] / new_graf_week['PLAN_WEEK']).mul(100).round(2).astype(str).add(' %')

In [171]:
new_graf_week['%TOTAL'] =( new_graf_week['TOTAL_FAKT'] / new_graf_week['TOTAL_PLAN']).mul(100).round(2).astype(str).add(' %')

In [172]:
new_graf_week.rename(columns={'NAME':f'WORK on {pd.to_datetime(start_week).date()}  to {pd.to_datetime(finish_week).date()}'}, inplace=True)

In [173]:
worksheet_SMR.worksheet('PLAN_WEEK').clear()
worksheet_SMR.worksheet('PLAN_WEEK').append_rows([new_graf_week.columns.values.tolist()]+new_graf_week.values.tolist())

{'spreadsheetId': '1ngWR2k38yG5qDWfwA9LKi4-R9q9PrIHludYb5vSh4Gc',
 'updates': {'spreadsheetId': '1ngWR2k38yG5qDWfwA9LKi4-R9q9PrIHludYb5vSh4Gc',
  'updatedRange': 'PLAN_WEEK!A1:I89',
  'updatedRows': 89,
  'updatedColumns': 9,
  'updatedCells': 801}}

In [ ]:
# справочно
dir(worksheet_SMR.worksheet('PLAN_WEEK'))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_finder',
 '_properties',
 'acell',
 'add_cols',
 'add_rows',
 'append_row',
 'append_rows',
 'batch_get',
 'batch_update',
 'cell',
 'clear',
 'clear_basic_filter',
 'client',
 'col_count',
 'col_values',
 'delete_row',
 'delete_rows',
 'duplicate',
 'export',
 'find',
 'findall',
 'format',
 'freeze',
 'frozen_col_count',
 'frozen_row_count',
 'get',
 'get_all_records',
 'get_all_values',
 'id',
 'insert_row',
 'merge_cells',
 'range',
 'resize',
 'row_count',
 'row_values',
 'set_basic_filter',
 'sort',
 'spreadsheet',
 'title',
 'update',
 'update_acell',
 'update_cell',
 'update_cells',
 'update_index',
 'update_tit

## данные по стартовым работам -> триггер для Plan Radar

In [174]:
start = start_project
finish = pd.to_datetime(pd.Timestamp.today().date())
start, finish

('2022-02-01', Timestamp('2023-03-20 00:00:00'))

In [ ]:
finish_yesterday = finish - pd.Timedelta(1, "d")

In [ ]:
total_plan = graf_total.loc[:, idx[start:finish,'план']].sum(axis=1).values
cum_sum_yesterday = graf_total.loc[:, idx[start:finish_yesterday,'план']].sum(axis=1).values
sum_today = graf_total.loc[:, idx[finish:finish,'план']].sum(axis=1).values
# total_plan_value = graf.loc[:, idx[start:finish,'план']].sum(axis=1).values - graf.loc[:, idx[start:finish,'план']].sum(axis=1).values

In [ ]:
total_plan = pd.DataFrame(total_plan,index=graf_total.index, columns=['cum_sum_today'])
cum_sum_yesterday = pd.DataFrame(cum_sum_yesterday,index=graf_total.index, columns=['cum_sum_yesterday'])
sum_today = pd.DataFrame(sum_today,index=graf_total.index, columns=[f'sum_today {pd.to_datetime(finish).date()}'])
total_plan = total_plan.join([cum_sum_yesterday, sum_today])

In [ ]:
total_plan = total_plan[(total_plan[f'sum_today {pd.to_datetime(finish).date()}']>0) & (total_plan['cum_sum_yesterday'] == 0)]

In [ ]:
total_plan = total_plan[[f'sum_today {pd.to_datetime(finish).date()}']]

In [ ]:
worksheet_SMR.worksheet('START').clear()
gd.set_with_dataframe(worksheet_SMR.worksheet('START'), total_plan,row=1, col=1, include_index=True, include_column_header=True)

## обновление листа ПЛАН-РАБОТ-> внесение факта

In [ ]:
# list_date = list(graf_black.iloc[0, :].values)
# list_name = list(graf_black.iloc[:, 0].values)

In [ ]:
# for i in plan_PR.iloc[:, 0]:
#     index_in_plan_PR = list(plan_PR.iloc[:, 0]).index(i)
#     date_in_plan_PR = plan_PR.iloc[index_in_plan_PR, 2]
#     date_in_plan_PR = '.'.join(reversed(str(date_in_plan_PR).split('-')))
#     value_in_plan_PR = plan_PR.iloc[index_in_plan_PR, 1]
#     value_in_plan_PR = ','.join(value_in_plan_PR.split('.'))

#     index_row_in_graf_black = list_name.index(i) + 2
#     index_column_in_graf_black = list_date.index(date_in_plan_PR) + 1

#     worksheet_SMR.worksheet("ГРАФИК РАБОТ").update_cell(index_row_in_graf_black, index_column_in_graf_black, value_in_plan_PR)

#     print(value_in_plan_PR, date_in_plan_PR, i, index_row_in_graf_black, index_column_in_graf_black)

## обновление листа финансирование:

1. внесение плана КС, кумулятивно за отчетный период
2. акутулизация плана платежей
3. внесение сумм по фактически выполненным работам

## графики:

1. СМР план/факт
2. финансирование план/факт
3. освоение план/факт


In [ ]:
from pygsheets.chart import ChartType

In [ ]:
gz = pygsheets.authorize(service_file='/content/credentials.json')

In [ ]:
 gz = gz.open('План работ Договор Адамант')

In [ ]:
gz.worksheet_by_title('PLAN_WEEK').add_chart(('A2', 'A44'), [('B2', 'B44')], chart_type=ChartType.LINE, title='MyChart', anchor_cell='A50')

<Chart LINE 'MyChart'>

# **Обновление графика Закачика**

* позиции имеющие шифр обновляются по шифру
* позиции без шифра обновляются после фильтрации по шифру

## предобработка графика Заказчика

In [175]:
#  список из графика заказчика
list2 = df_graph['Unnamed: 13']
list2 = list2.map(lambda z: str(z).lstrip())
list2 = list2.to_list()
list3 = df_graph['Unnamed: 2']
list3 = list3.map(lambda z: str(z).lstrip())
list3 = list3.to_list()

## обновление СМР

* пайплайн по каждому контракту СМР одинаковый

### пересечение списков видов работ

In [ ]:
#  список из шиифров из 'График работ'
list1 = graf.iloc[:, 0].values
list1

In [ ]:
result = list(set(list1) & set(list2))
result

In [178]:
for i in result:
    df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 4'] = (pd.to_numeric(graf.loc[graf.iloc[:,0].map(lambda z: str(z).lstrip()).values == i, 9].values[0].replace(',', '.').replace('%', ''))/100)
    # проверка корректности внесения обновлений
    print({df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 2'].values[0]: pd.to_numeric(graf.loc[graf.iloc[:,0].map(lambda z: str(z).lstrip()).values == i, 9].values[0].replace(',', '.').replace('%', ''))/100})    

{'               Устройство щебеночного основания': nan}
{'                  8 захватка': 0.8409}
{'         Монтаж молниезащиты ': nan}
{'            Изготовление и монтаж информационного стенда': 1.0}
{'               Укладка плит ПАГ -14 2 ряд': nan}
{'               Устройство вертикальных конструкций в осях 16-18_А-И': nan}
{'            Водоснабжение и водоотведение': nan}
{'               Монтаж башенного крана №1': 1.0}
{'         Устройство фасада ': nan}
{'            Выдача рабочего проекта по разделу : Вертикальные конструкции 5 этаж': nan}
{'            Выдача рабочего проекта по разделу . Вертикальные конструкции': nan}
{'               Выдача рабочего проекта по разделу': nan}
{'               Армирование фундаментной плиты ': 1.0}
{'            Выдача рабочего проекта по разделу': nan}
{'               Устройство плиты перекрытия в осях 16-18_А-И': 0.22920000000000001}
{'               Монтаж стоек с устройством фундаментов в осях 16-18/А-Г': 1.0}
{'            Демонтаж

## обновление проектирования

* первый этап - обновить по шифрам
* второй этап - обновить по наименованию работ

In [179]:
perevod = {
    'STATUS A': 1,
    'STATUS B': 0.85,
    'STATUS C': 0.6,
    'WORK': 0.3,
    'START': 0.05,
    'HOLD': 0,
    '': ''
}

In [180]:
list_PD_ID_1 = graph_PD[graph_PD.iloc[:, 1].values != ''].iloc[:,2]
list_PD_name = graph_PD[graph_PD.iloc[:, 1].values == ''].iloc[:,3]
list_PD = list_PD_ID_1.append(list_PD_name)
list_PD

<ipython-input-180-b66fa13cbffa>:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  list_PD = list_PD_ID_1.append(list_PD_name)


77                                         608AR-20-Р-ГП
83                                         608AR-20-Р-АР
86                                        608AR-20-Р-АР1
87                                        608AR-20-Р-АР2
88                                        608AR-20-Р-АР3
                             ...                        
243    Первчиное консультирование по СМР проведение т...
244                         Сопровождение строительства 
245                Отчет по вводу объекта в эксплуатацию
246                        Энергетическое моделирование 
247                                      Итоговый аудит 
Length: 248, dtype: object

In [181]:
result_PD_ID = list(set(list_PD) & set(list2))
result_PD_name = list(set(list_PD) & set(list3))

In [182]:
for i in result_PD_ID:
    df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 4'] = perevod[graph_PD.loc[graph_PD.iloc[:,2] == i,7].values[0]]
    # проверка корректности внесения обновлений
    print({df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 2'].values[0]: perevod[graph_PD.loc[graph_PD.iloc[:,2] == i,7].values[0]]})

{'            Наружное электроснабжение': 0.85}
{'            Наружные сети связи': 0}
{'            Гидроизоляция подземных конструкций': 1}
{'            Система позиционирования': 0.3}
{'            Лестницы надземной части': 1}
{'            Автоматическая установка водяного пожаротушения. Внутренний противопожарный водопровод. Насосная станция противопожарного водоснабжения': 0}
{'            Локальная вычислительная сеть': 0.3}
{'            Вертикальный транспорт': 0.05}
{'            Система видеонаблюдения': 0.6}
{'            Общие требования к ИТ-инфраструктуре': 1}
{'            Подземная часть. Вертикальные конструкции. 608AR-20-Р-КЖ-1-1': 1}
{'            Лестницы подвала': 1}
{'            Плиты покрытия 608AR-20-Р-КЖ-12-1': 0}
{'            Плита перекрытия над третьим этажом 608AR-20-Р-КЖ-8-1': 0.6}
{'            Отопление и внутреннеее теплоснабжение': 0.3}
{'            Индивидуальный тепловой пункт. Электрооборудование и автоматизация': 0}
{'            Благоустройс

In [183]:
for i in result_PD_name:
    df_graph.loc[df_graph['Unnamed: 2'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 4'] = perevod[graph_PD.loc[graph_PD.iloc[:,3] == i,7].values[0]]
    print({df_graph.loc[df_graph['Unnamed: 2'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 2'].values[0]: perevod[graph_PD.loc[graph_PD.iloc[:,3] == i,7].values[0]]}) 

{'            Система мультимедиа': 0.85}
{'               Слаботочные системы': 0.85}
{'         Мультимедиа, IT': ''}
{'            Интерьеры. Детали отделки': 0.3}
{'            Однолинейные схемы ВРУ': 0}
{'            Интерьеры. Полы': 0.3}
{'               Система водоотведения': 0.85}
{'               Предоставление ВОР конструктивной части здания ниже 0,000': 1}
{'            Общие требования к ИТ-инфраструктуре': 0.85}
{'         Дизайн-проект интерьеров': 1}
{'               Общая пояснительная записка': 0.85}
{'            Задание для раздела ЭОМ': 0.85}
{'            Эскизный проект АИ: План и спецификация витражей и дверей': 0}
{'            Генеральный план': 0.85}
{'            Информационная безопасность': 0.85}
{'            Итоговый проект АИ: План и спецификация витражей и дверей': 0}
{'            Разработка визуализации фасадов*': 0.85}
{'         Согласование Заказчиком рабочей документации с ресурсоснабжающими (сетевыми) организациями': 0}
{'         Инженерное о

создание файла с обновленными данными

In [184]:
df_graph.to_excel('Сводный план-график {}.xlsx'.format('20.03.2023'), sheet_name='Обновление от {}'.format('20.03.2023'),index=False)